In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

def load_dataset_from_npz(npz_path, test_size=0.2):
    data = np.load(npz_path)
    x = data['arr_0']
    y = data['arr_1']
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=42)
    return x_train, x_test, y_train, y_test


In [ ]:
import tensorflow as tf

def create_model(input_shape, num_classes):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(input_shape,)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model


In [1]:
pip install tensorflow flwr

Note: you may need to restart the kernel to use updated packages.


In [3]:
import argparse
import numpy as np
from flwr.client import ClientApp, NumPyClient
import tensorflow as tf
import dataset
import model as model_module

# Make TensorFlow log less verbose
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Define the path to your .npz dataset
npz_path = r"C:\Users\aldri\federatedd\dataset\CpE_Faculty_Members.npz"

# Load dataset
x_train, x_test, y_train, y_test = dataset.load_dataset_from_npz(npz_path, test_size=0.2)

# Encode labels as integers (this part should be added here if needed)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Partition the dataset
def partition_data(x, y, num_partitions):
    partition_size = len(x) // num_partitions
    partitions = []
    for i in range(num_partitions):
        start = i * partition_size
        end = start + partition_size
        partitions.append((x[start:end], y[start:end]))
    return partitions

train_partitions = partition_data(x_train, y_train, 2)
test_partitions = partition_data(x_test, y_test, 2)

# Print dataset shapes after partitioning
print("Training data shapes after partitioning:", [(x.shape, y.shape) for x, y in train_partitions])
print("Testing data shapes after partitioning:", [(x.shape, y.shape) for x, y in test_partitions])

# Verify the content of the partitions (optional, for detailed verification)
print("Training partition 0 labels:", np.unique(train_partitions[0][1], return_counts=True))
print("Training partition 1 labels:", np.unique(train_partitions[1][1], return_counts=True))
print("Testing partition 0 labels:", np.unique(test_partitions[0][1], return_counts=True))
print("Testing partition 1 labels:", np.unique(test_partitions[1][1], return_counts=True))

# Define Flower client
class FlowerClient(NumPyClient):
    def __init__(self, x_train, y_train, x_val, y_val, model):
        super().__init__()
        self.x_train = x_train
        self.y_train = y_train
        self.x_val = x_val
        self.y_val = y_val
        self.model = model

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(
            self.x_train,
            self.y_train,
            validation_data=(self.x_val, self.y_val),
            epochs=10,
            batch_size=32,
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)]
        )
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.x_val, self.y_val)
        return loss, len(self.x_val), {"accuracy": accuracy}

def client_fn(cid: str, partition_id: int):
    """Create and return an instance of Flower `Client`."""
    # Load the partitioned data based on the partition ID
    x_partition, y_partition = train_partitions[partition_id]
    x_val, y_val = test_partitions[partition_id]
    # Create model instance for the client
    num_classes = len(np.unique(y_train))  # Number of unique classes (i.e., number of persons)
    input_shape = x_train.shape[1]  # Number of features
    dense_model = model_module.create_model(input_shape, num_classes)
    dense_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return FlowerClient(x_partition, y_partition, x_val, y_val, dense_model).to_client()



# Flower ClientApp
app = ClientApp(
    client_fn=client_fn,
)

# Legacy mode
if __name__ == "__main__":
    from flwr.client import start_client
    parser = argparse.ArgumentParser()
    parser.add_argument("--client_id", type=str, help="Client ID")
    parser.add_argument("--partition_id", type=int, help="Partition ID")
    args = parser.parse_args()

    start_client(
        server_address="192.168.1.10:8080",
        client=client_fn(args.client_id, args.partition_id),
    )


Training data shapes after partitioning: [((929, 512), (929,)), ((929, 512), (929,))]
Testing data shapes after partitioning: [((232, 512), (232,)), ((232, 512), (232,))]
Training partition 0 labels: (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64), array([ 60,  68, 104,  77,  81,  68,  89, 137,  93,  72,  80], dtype=int64))
Training partition 1 labels: (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64), array([ 65,  51,  91,  79,  88,  84,  81, 157,  69,  73,  91], dtype=int64))
Testing partition 0 labels: (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([12, 18, 26, 22, 17, 19, 20, 36, 22, 18, 22], dtype=int64))
Testing partition 1 labels: (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([14, 25, 29, 16, 19, 19, 23, 36, 13, 13, 25], dtype=int64))


usage: ipykernel_launcher.py [-h] [--client_id CLIENT_ID] [--partition_id PARTITION_ID]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\aldri\AppData\Roaming\jupyter\runtime\kernel-50b25313-ff1b-401d-9092-67135b374829.json


SystemExit: 2

C:\Users\aldri\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
